# **Asynchronous Programming in Python**

This is the first of two classes on concurrent programming in Python. In this session, we'll focus on asynchronous programming, while the next class will cover multithreading and multiprocessing.

**Table of Contents**

- [Problem: Blocking Operations](#scrollTo=AWI1a7rl77sN)
- [Synchronous vs Asynchronous](#scrollTo=JZ16CoowTHyL)
- [Concurrency, Parallelism, and Coroutines](#scrollTo=OyXHgwifCHvr)
- [`async`, `await`, and `asyncio`](#scrollTo=b7Jmexer5jjv)
- [Exercise: Asynchronous API Calls](#scrollTo=Lt4pMMM6LbI9)


## **Problem: Blocking Operations**

Blocking operations pause the execution of a program until they are fully completed. When these operations occur, no other tasks can be processed until the blocking operation finishes. These operations are usually categorized as either **CPU-bound** or **I/O-bound**.

- **CPU-bound tasks**

  Tasks that spend most of the time performing calculations. They are limited by the processing power of your central processing unit (CPU).

- **I/O-bound tasks**

  Tasks that spend most of the time waiting on input/output operations. They are limited by the speed of data transfer between external systems.

> In this class, we'll focus on handling I/O-bound tasks, while we will cover CPU-bound tasks in the next session.

<br>

### **Common I/O-bound operations**:

**File operations**
-	Reading or writing large files from/to a disk.
-	Copying or moving files between directories or storage devices.

**Network communication**
-	Downloading/uploading files over the internet.
-	Making HTTP requests to APIs or websites.
-	Real-time streaming (audio/video) where the program waits for data packets.

**Database interactions**
-	Querying a remote database (e.g., fetching records from a cloud database).
-	Inserting, updating, or deleting records in a database.

**User input/output**
-	Waiting for user input through a keyboard or mouse.
-	Processing clicks, form submissions, or other user events in a GUI application.

**Device I/O**
-	Interacting with external devices, like reading data from a USB drive, sensors, or a camera.
-	Communicating with hardware components such as hard drives or peripheral devices.


### **Example 1: Simulate Database Interactions**

Consider this example of getting user information from a database for a social media:

In [ ]:
# Libraries to time our code and simluate blocking operations
import time
import random

In [ ]:
def fetch_from_database():
  print("Fetching user data from database...")
  print("---")
  time.sleep(4)  # Simulating a slow operation
  return {"id": 12, "name": "Tommy", "followers": 100, "following": 35}


def fetch_user_posts():
  print("Fetching user posts...")
  print("---")
  time.sleep(3)  # Simulating a slow operation
  return {"post1": "I am a teapot!", "post2": "What happend to my ice cream :(", "post3": "I've been waiting in a line for 3hrs!"}


def fetch_notifications():
  print("Fetching notifications...")
  print("---")
  time.sleep(2)  # Simulating a slow operation
  return ["New post", "New follower", "New friend request"]


def get_user_data():
  user = fetch_from_database()          # Takes 4 second
  posts = fetch_user_posts()            # Takes 3 second
  notifications = fetch_notifications() # Takes 2 second
  return user, posts, notifications     # Total: 9 seconds!


# Execute our code and time it
start_time = time.perf_counter()
user, posts, notifications = get_user_data()
end_time = time.perf_counter()
print("User information:")
print(user)
print(posts)
print(notifications)
print("---")

# Log the time it takes to finish operation
print(f"Finished in {end_time - start_time:.2f} seconds")

Fetching user data from database...
---
Fetching user posts...
---
Fetching notifications...
---
User information:
{'id': 12, 'name': 'Tommy', 'followers': 100, 'following': 35}
{'post1': 'I am a teapot!', 'post2': 'What happend to my ice cream :(', 'post3': "I've been waiting in a line for 3hrs!"}
['New post', 'New follower', 'New friend request']
---
Finished in 9.01 seconds


### **Example 2: Simulate Multiple HTTP Requests**

Or this example of performing multiple HTTP requests within a loop:

In [ ]:
# This is to control our simulation
random.seed(0)


def sim_request(n):
  print(f"Starting processing task: {n}")

  # Pick random int between 2 to 4 to simulate a blocking operation
  delay = random.randint(2, 4)
  print(f"Process time: {delay} second(s)")

  # Randomly delay between 2 to 4 seconds to simulate the time it takes to wait for a http response
  time.sleep(delay)
  print(f"Finished processing task: {n}")


def make_n_requests(n):
  # Perform the blocking task n times
  for i in range(1, n + 1):
    sim_request(i)
    print("---")


# Execute our code and time it
start_time = time.perf_counter()
make_n_requests(3)
end_time = time.perf_counter()

# Log the time it takes to finish operation
print(f"Finished in {end_time - start_time:.2f} seconds")

Starting processing task: 1
Process time: 3 second(s)
Finished processing task: 1
---
Starting processing task: 2
Process time: 3 second(s)
Finished processing task: 2
---
Starting processing task: 3
Process time: 2 second(s)
Finished processing task: 3
---
Finished in 8.01 seconds


In the first example, the program has to perform back-to-back database interactions. In the second example, the program has to perform multiple http requests within a loop. These are really slow because database interactions and http requests are blocking operations and while the blocking operation is happening, the rest of the program is just waiting for those operations to finish. To solve the problem of our program stalling, we can leverage Python **coroutines** (asynchronous programming) to write code that are more efficient.

## **Synchronous vs Asynchronous**

In traditional programming, operations happen one after another. This is called **synchronous programming** (such as the examples above). When you encounter a slow operation, your program waits for it to finish before moving on. This can lead to performance bottlenecks if the operations take a long time to complete.

To help Alleviate this problem, we can use python **coroutines** to start multiple tasks **concurrently** and gather the results when they are done without having to wait to perform other tasks (Sort of like a chef who starts the rice cooker, then chops vegetables and prepare seasoning while the rice cooks). Python coroutines are what we refer to as **asynchronous programming**, which can greatly speed up your programs.

<figure align="center">
  <img src="https://raw.githubusercontent.com/kchenTTP/python-async/5f0c90201af6e2631ef11093583239519403c8bb/assets/sync-vs-async.png" alt="sync-vs-async.png" />
  <figcaption>Synchronous vs asynchronous</figcaption>
</figure>


## **Concurrency, Parallelism, and Coroutines**

Before we optimize the examples above for speed, let's first understand the difference between **concurrency** and **parallelism**, as well as **coroutines** in Python.

<br>

#### **Concurrency**

In computer science, concurrency means that multiple functions or tasks are making progress at the same time, but not necessarily simultaneously *([Wikipedia](https://en.wikipedia.org/wiki/Concurrency_(computer_science)))*. This is not to be confused with parallelism, which means making progress simultaneously. Both concurrency and parallelism are ways to achieve multitasking.

#### **Parellelism**

Parallelism refers to the ability to execute multiple tasks or calculations simultaneously by utilizing multiple processors or CPU cores. Unlike concurrency, which manages multiple tasks within a single processor, parallelism actually performs tasks at the exact same time on different processors.

<br>

> In Python, multitasking can be achieved in several ways:
>
> - Coroutines (`asyncio`): Achieves concurrency using an event loop in a single thread
> - Threads (`concurrent.futures.ThreadPoolExecutor` & `threading`): Allows concurrent execution using multiple threads, but not true parallelism due to Python's Global Interpreter Lock (GIL)
> - Processes (`concurrent.futures.ProcessPoolExecutor` & `multiprocessing`): Achieves parallel execution by running tasks in separate processes
>
> For this course, we'll only focus on **coroutines**.

<br>

#### **Coroutines**

Coroutines are functions that can pause and resume. They use an event loop to switch between tasks without blocking the program. Although tasks make progress concurrently, they don't run simultaneously (parallelism). In Python, coroutines are created with the `async` and `await` keyword to allow long-running tasks to yield control, so other tasks can run without waiting.

<br>

<figure align="center">
  <img src="https://raw.githubusercontent.com/kchenTTP/python-async/5f0c90201af6e2631ef11093583239519403c8bb/assets/event-loop.png" alt="event-loop.png" />
  <figcaption>Python event loop</figcaption>
</figure>


## **async, await, and asyncio**

To creat asynchronous programs in python, we can leverage the `async` and `await` keyword as well as the `asyncio` library.

**`async`**

The `async` keyword tells Python that a function is a coroutine. Think of it as a "pausable" function.

**`await`**

The `await` keyword is like a "pause and wait" sign. It tells Python to pause the function until the awaited operation completes.

> 🚨 You can only use `await` inside async functions.


### **Optimize - Example 1: Simulate Database Interactions**

Now we can optimize the [first example](#scrollTo=8jnPXNon75gI) by making the function asynchronous using `async` and `await`.

Let's import `asyncio` first to use the asyncrhonous version of `time.sleep()`

In [ ]:
import asyncio

First, convert the function to an asynchronous one:
1. Add the `async` keyword in front of the function signature.
2. Add the `await` keyword before blocking operations (such as network calls, database interactions, file read...etc). In our code, `sleep()` is the blocking operation so we want to put await before `sleep()`.
3. Use the asynchronous version of `sleep()` since not all python functions and libraries support coroutines.

In [ ]:
async def async_fetch_from_database():
  print("Fetching user data from database...")
  await asyncio.sleep(4)
  return {"id": 12, "name": "Tommy", "followers": 100, "following": 35}


async def async_fetch_user_posts():
  print("Fetching user posts...")
  await asyncio.sleep(3)
  return {"post1": "I am a teapot!", "post2": "What happend to my ice cream :(", "post3": "I've been waiting in a line for 3hrs!"}


async def async_fetch_notifications():
  print("Fetching notifications...")
  await asyncio.sleep(2)
  return ["New post", "New follower", "New friend request"]

Great! We've successfully converted the database interaction functions to coroutines. We can stop here and Python won't complain. However, without an event loop (like the one from `asyncio`), you can define and await for coroutines, but they won't run concurrently.

Let's see how we can make our functions run concurrently using `asyncio` below.

**`asyncio`**

`asyncio` is a library in Python that provides an event loop as its core functionality to write concurrent code using the async/await syntax.

asyncio is used as a foundation for multiple Python asynchronous frameworks that provide high-performance network and web-servers, database connection libraries, distributed task queues, etc.

asyncio is often a perfect fit for IO-bound and high-level structured network code.

[Documentation](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather)

<br>

---

**Usage**

- `asyncio.gather()`: Run awaitable objects in the function argument sequence concurrently.

  > ❗Important: Without this `gather()` step, your program will not be executed asynchronously.

- `asyncio.run()`: Run the top-level entry point function (if no event loop is present)
  > 🚨 If an event loop is already running (such as in Google Colab), simply use `await` before the top-level entry point function.

In [ ]:
async def async_get_user_data():
  user, posts, notifications = await asyncio.gather(async_fetch_from_database(), async_fetch_user_posts(), async_fetch_notifications())
  return user, posts, notifications

Now lets test our optimized code to see if it's running any faster

In [ ]:
# Execute our code and time it
start_time = time.perf_counter()

# Use this only in Colab or Jupyter notebooks since there is already an existing event loop running in colab
user, posts, notifications = await async_get_user_data()

# In your python script files (.py) comment out the above line and use this line below instead
# asyncio.run(get_user_data())

print("---")

end_time = time.perf_counter()
print("User information:")
print(user)
print(posts)
print(notifications)
print("---")

# Log the time it takes to finish operation
print(f"Finished in {end_time - start_time:.2f} seconds")

Fetching user data from database...
Fetching user posts...
Fetching notifications...
---
User information:
{'id': 12, 'name': 'Tommy', 'followers': 100, 'following': 35}
{'post1': 'I am a teapot!', 'post2': 'What happend to my ice cream :(', 'post3': "I've been waiting in a line for 3hrs!"}
['New post', 'New follower', 'New friend request']
---
Finished in 4.00 seconds



<div align="center">
<img src="https://i.pinimg.com/originals/84/df/28/84df2849675395d4acafa8859e6813bd.gif" alt="minions celebration gif" style=/>
<figcaption>Minions celebrating</figcaption>
</div>

We've successfully optimized our code! The time it takes to run our program now takes around 4 seconds to finish as opposed to the original 9 seconds! That's less than half the amount of the original time!


### **Optimize - Example 2: Simulate Multiple HTTP Requests**

We can also optimize the [second example](#scrollTo=TGrnvACmHU9J) using `async`, `await`, and `asyncio`

In [ ]:
# This is to control our simulation
random.seed(0)


async def async_sim_request(n):
  print(f"Starting processing task: {n}")

  # Pick random int between 2 to 4
  delay = random.randint(2, 4)
  print(f"Process time: {delay} second(s)")

  # Randomly delay between 2 to 4 seconds to simulate the time it takes to perform tasks
  await asyncio.sleep(delay)
  print(f"Finished processing task: {n}")


async def async_make_n_requests(n):
  tasks = []

  # Instead of performing the task right away, first create a list of tasks
  for i in range(1, n + 1):
    tasks.append(async_sim_request(i))

  # Then afterwards execute all the tasks asynchronously at once
  await asyncio.gather(*tasks)



# Execute our code and time it
start_time = time.perf_counter()

# Use this only in Colab or Jupyter notebooks since there is already an existing event loop running in colab
await async_make_n_requests(3)

# In your python script files (.py) comment out the above line and use this line below instead
# asyncio.run(async_do_n_tasks(3))

print("---")

end_time = time.perf_counter()

# Log the time it takes to finish operation
print(f"Finished in {end_time - start_time:.2f} seconds")

Starting processing task: 1
Process time: 3 second(s)
Starting processing task: 2
Process time: 3 second(s)
Starting processing task: 3
Process time: 2 second(s)
Finished processing task: 3
Finished processing task: 1
Finished processing task: 2
---
Finished in 3.00 seconds



<div align="center">
<img src="https://y.yarn.co/d391ed43-a8b9-46c5-83f8-f81ed05c58ab_text.gif" alt="hip hip hooray gif" style=/>
<figcaption>Hip hip hooray!!!</figcaption>
</div>

We've successfully optimized our code again! It now takes around 3 seconds to run our code instead of the original 8 seconds!

## **Exercise: Asynchronous API Calls**

In this exercise, we will be speeding up multiple API calls using coroutines.

**Scenario**

You're interested in multiple [NASA API](https://api.nasa.gov/)'s Astronomy Picture of the Day (APOD) from the past year but waiting for all of them takes a while. How can we optimize our code so we don't have to wait forever?

### Install Asynchronous HTTP Library
`requests` cannot leverage the asynchronous capabilities of python so we'll need to use another library that can: `httpx`

> Note: In the next class, we'll explore multithreading as an alternative solution for libraries that don't fully support Python's coroutine features.



In [ ]:
!pip install httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


### Import Libraries

- `httpx`: Asynchronous HTTP library that handles async requests
- `requests`: Synchronous HTTP library
- `typing`: For type hints
- `dataclasses`: Classes purely for storing data
- `datetime`: Date and time library
- `IPython.display`: Display images
- `rich`: For rich text formatting
- `google.colab.userdata`: For Colab Secrets management

In [ ]:
import httpx
import requests

from typing import Generator
from dataclasses import dataclass
import datetime

from IPython.display import Image, display
from rich.pretty import pprint

from google.colab import userdata

### Store API Key in Colab Secrets

See the 🔑 icon on the left side of your Colab notebook? That is where you should store your secrets. Copy your api key that you got from NASA API and store it there. Give it a name of `nasa_key`.

We can then access the api key safely in our code by running the below cell

> 🚨 Never store your api keys directly inside your code as a string. It is extremely unsafe.

In [ ]:
# NASA API key
nasa_key = userdata.get('nasa_key')

Store the APOD API endpoint

In [ ]:
# store the astronomy picture of the day api endpoint
apod_endpoint = 'https://api.nasa.gov/planetary/apod'

### Helper Functions

Let's write some helper functions to help us with:
- storing the APOD information
- getting the first day of each month in a year
- constructing the api endpoints with parameters
- displaying the apod

In [ ]:
@dataclass
class APOD:
  title: str
  date: str
  explanation: str
  url: str


def first_day_of_months_generator(year: int) -> Generator:
  for month in range(1, 13):
    yield datetime.date(year, month, 1).strftime('%Y-%m-%d')


def create_endpoint(endpoint: str, api_key: str, **params) -> str:
  endpoint = endpoint + "?"
  for k, v in params.items():
    endpoint = endpoint + f"{k}={v}&"
  endpoint = endpoint + f"api_key={api_key}"
  return endpoint


def display_apods(apods: list[APOD]) -> None:
  import textwrap  # For wrapping text

  for apod in apods:
    print(f"Title: {apod.title}")
    print(f"Date: {apod.date}")
    print(f"Explanation: {textwrap.fill(apod.explanation, width=80)}")
    display(Image(url=apod.url, width=700))
    print("---")

### Sync API Calls

Now let's first try using the `request` library (synchronous) to make the API calls

In [ ]:
# Create a function that makes the synchronous api calls
def get_apods(dates: Generator, endpoint: str, api_key: str, **params):
  result = []
  with requests.Session() as session:
    for date in dates:
      pprint(f"Getting APOD for: {date}")
      url = create_endpoint(endpoint, api_key, date=date, **params)
      resp = session.get(url)
      resp.raise_for_status()
      result.append(
          APOD(
              title=resp.json()['title'],
              date=resp.json()['date'],
              explanation=resp.json()['explanation'],
              url=resp.json()['url']
          )
      )
  return result


# Call and time our function
start_time = time.perf_counter()

results = get_apods(first_day_of_months_generator(2023), apod_endpoint, nasa_key, thumbs=True)
print("---")
pprint(results)

print("---")
pprint(f"Finished in {time.perf_counter() - start_time:.2f} seconds")

'Getting APOD for: 2023-01-01'

'Getting APOD for: 2023-02-01'

'Getting APOD for: 2023-03-01'

'Getting APOD for: 2023-04-01'

'Getting APOD for: 2023-05-01'

...

'Getting APOD for: 2023-12-01'

---


[
│   APOD(
│   │   title='The Largest Rock in our Solar System',
│   │   date='2023-01-01',
│   │   explanation="There, that dot on the right, that's the largest rock known in our Solar System. It is larger than every known asteroid, moon, and comet nucleus.  It is larger than any other local rocky planet.  This rock is so large its gravity makes it into a large ball that holds heavy gases near its surface.  (It used to be the largest known rock of any type until the recent discoveries of large dense planets orbiting other stars.)  The Voyager 1 spacecraft took the featured picture -- famously called Pale Blue Dot -- of this giant space rock in 1990 from the outer Solar System.  Today, this rock starts another orbit around its parent star, for roughly the 5 billionth time, spinning over 350 times during each trip.  Happy Gregorian Calendar New Year to all inhabitants of this rock we call Earth.",
│   │   url='https://apod.nasa.gov/apod/image/2301/PaleBlueDotOrig_Voyager1_960.jpg'
│   ),
│   APOD(
│   │   title='The Seventh World of Trappist-1',
│   │   date='2023-02-01',
│   │   explanation="Seven worlds orbit the ultracool dwarf star TRAPPIST-1. A mere 40 light-years away, many of the exoplanets were discovered in 2016 using the Transiting Planets and Planetesimals Small Telescope (TRAPPIST) located at  La Silla Observatory in Chile and later confirmed with telescope including NASA's Spitzer Space Telescope. The TRAPPIST-1 planets are likely all rocky and similar in size to Earth, and so compose one of the largest treasure troves of terrestrial planets ever detected around a single star. Because they orbit very close to their faint, tiny star they could also have regions where surface temperatures allow for the presence of ice or even liquid water, a key ingredient for life. Their tantalizing proximity to Earth makes them prime candidates for future telescopic explorations of the atmospheres of potentially habitable planets.  All seven exoplanets appear in the featured illustration, which imagines a view from the most distant known world of this system, TRAPPIST-1h, as having a rocky landscape covered in ice. Meanwhile, in the imagined background, one of the system's inner planets crosses in front of the dim, orange, nearly Jupiter-sized parent star.   Astrophysicists: Browse 3,000+ codes in the Astrophysics Source Code Library",
│   │   url='https://apod.nasa.gov/apod/image/2302/Trappist1h_Carroll_960.jpg'
│   ),
│   APOD(
│   │   title='The Flaming Star Nebula',
│   │   date='2023-03-01',
│   │   explanation="Is star AE Aurigae on fire? No.  Even though AE Aurigae is named the Flaming Star and the surrounding nebula IC 405 is named the Flaming Star Nebula, and even though the nebula appears to some like a swirling flame, there is no fire.  Fire, typically defined as the rapid molecular acquisition of oxygen, happens only when sufficient oxygen is present and is not important in such high-energy, low-oxygen environments such as stars.  The bright star AE Aurigae occurs near the center of the Flaming Star Nebula and is so hot it glows blue, emitting light so energetic it knocks electrons away from surrounding gas. When a proton recaptures an electron, light is emitted, as seen in the surrounding emission nebula.  Captured here three weeks ago, the Flaming Star Nebula is visible near the composite image's center, between the red Tadpole Nebula on the left and blue-tailed Comet ZTF on the right.  The Flaming Star Nebula lies about 1,500 light years distant, spans about 5 light years, and is visible with a small telescope toward the constellation of the Charioteer (Auriga).",
│   │   url='https://apod.nasa.gov/apod/image/2303/FlamingStarComet_Roell_1080.jpg'
│   ),
│   APOD(
│   │   title='NGC 2442: Galaxy in Volans',
│   │   date='2023-04-01',
│   │   explanation="Distorted galaxy NGC 2442 can be found in the southern constellation of the flying fish, (Piscis) Volans. Located about 50 million light-years away, the galaxy's two

---


'Finished in 2.50 seconds'

### Async API Calls

Then we try using the `httpx` library (asynchronous) to make the API calls

In [ ]:
# Create functions that do asynchronous api calls
async def get_apod(date: str, endpoint: str, api_key: str, **params):
  url = create_endpoint(endpoint, api_key, **params)
  async with httpx.AsyncClient() as client:
    resp = await client.get(url, params={**params, 'date': date, 'api_key': api_key})
    resp.raise_for_status()
    return APOD(
        title=resp.json()['title'],
        date=resp.json()['date'],
        explanation=resp.json()['explanation'],
        url=resp.json()['url']
    )


async def get_apods_from_gen(dates: Generator, endpoint: str, api_key: str, **params):
  tasks = []
  for date in dates:
    pprint(f"Getting APOD for: {date}")
    tasks.append(get_apod(date, endpoint, api_key))
  return await asyncio.gather(*tasks)


# Call and time our functions
start_time = time.perf_counter()

# Use this only in Colab or Jupyter notebooks since there is already an existing event loop running in colab
apods = await get_apods_from_gen(first_day_of_months_generator(2023), apod_endpoint, nasa_key, thumbs=True)

# In your python script files (.py) comment out the above line and use this line below instead
# apods = asyncio.run(get_apods_from_gen(first_day_of_months_generator(2023), apod_endpoint, nasa_key, thumbs=True))

print("---")
pprint(apods)

print("---")
pprint(f"Finished in {time.perf_counter() - start_time:.2f} seconds")

'Getting APOD for: 2023-01-01'

'Getting APOD for: 2023-02-01'

'Getting APOD for: 2023-03-01'

'Getting APOD for: 2023-04-01'

'Getting APOD for: 2023-05-01'

...

'Getting APOD for: 2023-12-01'

---


[
│   APOD(
│   │   title='The Largest Rock in our Solar System',
│   │   date='2023-01-01',
│   │   explanation="There, that dot on the right, that's the largest rock known in our Solar System. It is larger than every known asteroid, moon, and comet nucleus.  It is larger than any other local rocky planet.  This rock is so large its gravity makes it into a large ball that holds heavy gases near its surface.  (It used to be the largest known rock of any type until the recent discoveries of large dense planets orbiting other stars.)  The Voyager 1 spacecraft took the featured picture -- famously called Pale Blue Dot -- of this giant space rock in 1990 from the outer Solar System.  Today, this rock starts another orbit around its parent star, for roughly the 5 billionth time, spinning over 350 times during each trip.  Happy Gregorian Calendar New Year to all inhabitants of this rock we call Earth.",
│   │   url='https://apod.nasa.gov/apod/image/2301/PaleBlueDotOrig_Voyager1_960.jpg'
│   ),
│   APOD(
│   │   title='The Seventh World of Trappist-1',
│   │   date='2023-02-01',
│   │   explanation="Seven worlds orbit the ultracool dwarf star TRAPPIST-1. A mere 40 light-years away, many of the exoplanets were discovered in 2016 using the Transiting Planets and Planetesimals Small Telescope (TRAPPIST) located at  La Silla Observatory in Chile and later confirmed with telescope including NASA's Spitzer Space Telescope. The TRAPPIST-1 planets are likely all rocky and similar in size to Earth, and so compose one of the largest treasure troves of terrestrial planets ever detected around a single star. Because they orbit very close to their faint, tiny star they could also have regions where surface temperatures allow for the presence of ice or even liquid water, a key ingredient for life. Their tantalizing proximity to Earth makes them prime candidates for future telescopic explorations of the atmospheres of potentially habitable planets.  All seven exoplanets appear in the featured illustration, which imagines a view from the most distant known world of this system, TRAPPIST-1h, as having a rocky landscape covered in ice. Meanwhile, in the imagined background, one of the system's inner planets crosses in front of the dim, orange, nearly Jupiter-sized parent star.   Astrophysicists: Browse 3,000+ codes in the Astrophysics Source Code Library",
│   │   url='https://apod.nasa.gov/apod/image/2302/Trappist1h_Carroll_960.jpg'
│   ),
│   APOD(
│   │   title='The Flaming Star Nebula',
│   │   date='2023-03-01',
│   │   explanation="Is star AE Aurigae on fire? No.  Even though AE Aurigae is named the Flaming Star and the surrounding nebula IC 405 is named the Flaming Star Nebula, and even though the nebula appears to some like a swirling flame, there is no fire.  Fire, typically defined as the rapid molecular acquisition of oxygen, happens only when sufficient oxygen is present and is not important in such high-energy, low-oxygen environments such as stars.  The bright star AE Aurigae occurs near the center of the Flaming Star Nebula and is so hot it glows blue, emitting light so energetic it knocks electrons away from surrounding gas. When a proton recaptures an electron, light is emitted, as seen in the surrounding emission nebula.  Captured here three weeks ago, the Flaming Star Nebula is visible near the composite image's center, between the red Tadpole Nebula on the left and blue-tailed Comet ZTF on the right.  The Flaming Star Nebula lies about 1,500 light years distant, spans about 5 light years, and is visible with a small telescope toward the constellation of the Charioteer (Auriga).",
│   │   url='https://apod.nasa.gov/apod/image/2303/FlamingStarComet_Roell_1080.jpg'
│   ),
│   APOD(
│   │   title='NGC 2442: Galaxy in Volans',
│   │   date='2023-04-01',
│   │   explanation="Distorted galaxy NGC 2442 can be found in the southern constellation of the flying fish, (Piscis) Volans. Located about 50 million light-years away, the galaxy's two

---


'Finished in 1.34 seconds'

As you can see, the asynchronous api calls are faster. Maybe not by a lot but we're only making 12 API calls here so there might not be a huge amount of difference at the moment. However, when the number of API calls go up, that's when the asynchronous version of our code will truly shine.

> 🚨 Just be careful you don't exceed the rate limit of the API and you should be fine doing this asynchronously.

<br>

Now let's take a look at all the beautiful Astronomy Picture of the Day we just got!

In [ ]:
display_apods(apods)

Title: The Largest Rock in our Solar System
Date: 2023-01-01
Explanation: There, that dot on the right, that's the largest rock known in our Solar System.
It is larger than every known asteroid, moon, and comet nucleus.  It is larger
than any other local rocky planet.  This rock is so large its gravity makes it
into a large ball that holds heavy gases near its surface.  (It used to be the
largest known rock of any type until the recent discoveries of large dense
planets orbiting other stars.)  The Voyager 1 spacecraft took the featured
picture -- famously called Pale Blue Dot -- of this giant space rock in 1990
from the outer Solar System.  Today, this rock starts another orbit around its
parent star, for roughly the 5 billionth time, spinning over 350 times during
each trip.  Happy Gregorian Calendar New Year to all inhabitants of this rock we
call Earth.


---
Title: The Seventh World of Trappist-1
Date: 2023-02-01
Explanation: Seven worlds orbit the ultracool dwarf star TRAPPIST-1. A mere 40 light-years
away, many of the exoplanets were discovered in 2016 using the Transiting
Planets and Planetesimals Small Telescope (TRAPPIST) located at  La Silla
Observatory in Chile and later confirmed with telescope including NASA's Spitzer
Space Telescope. The TRAPPIST-1 planets are likely all rocky and similar in size
to Earth, and so compose one of the largest treasure troves of terrestrial
planets ever detected around a single star. Because they orbit very close to
their faint, tiny star they could also have regions where surface temperatures
allow for the presence of ice or even liquid water, a key ingredient for life.
Their tantalizing proximity to Earth makes them prime candidates for future
telescopic explorations of the atmospheres of potentially habitable planets.
All seven exoplanets appear in the featured illustration, which imagines a v

---
Title: The Flaming Star Nebula
Date: 2023-03-01
Explanation: Is star AE Aurigae on fire? No.  Even though AE Aurigae is named the Flaming
Star and the surrounding nebula IC 405 is named the Flaming Star Nebula, and
even though the nebula appears to some like a swirling flame, there is no fire.
Fire, typically defined as the rapid molecular acquisition of oxygen, happens
only when sufficient oxygen is present and is not important in such high-energy,
low-oxygen environments such as stars.  The bright star AE Aurigae occurs near
the center of the Flaming Star Nebula and is so hot it glows blue, emitting
light so energetic it knocks electrons away from surrounding gas. When a proton
recaptures an electron, light is emitted, as seen in the surrounding emission
nebula.  Captured here three weeks ago, the Flaming Star Nebula is visible near
the composite image's center, between the red Tadpole Nebula on the left and
blue-tailed Comet ZTF on the right.  The Flaming Star Nebula lies about 

---
Title: NGC 2442: Galaxy in Volans
Date: 2023-04-01
Explanation: Distorted galaxy NGC 2442 can be found in the southern constellation of the
flying fish, (Piscis) Volans. Located about 50 million light-years away, the
galaxy's two spiral arms extending from a pronounced central bar give it a hook-
shaped appearance in this deep colorful image, with spiky foreground stars
scattered across the telescopic field of view. The image also reveals the
distant galaxy's obscuring dust lanes, young blue star clusters and reddish star
forming regions surrounding a core of yellowish light from an older population
of stars. But the star forming regions seem more concentrated along the drawn-
out (upper right) spiral arm. The distorted structure is likely the result of an
ancient close encounter with the smaller galaxy seen near the top left of the
frame. The two interacting galaxies are separated by about 150,000 light-years
at the estimated distance of NGC 2442.


---
Title: Carina Nebula North
Date: 2023-05-01
Explanation: The Great Carina Nebula is home to strange stars and iconic nebulas. Named for
its home constellation, the huge star-forming region is larger and brighter than
the Great Orion Nebula but less well known because it is so far south -- and
because so much of humanity lives so far north.  The featured image shows in
great detail the northernmost part of the Carina Nebula. On the bottom left is
the Gabriela Mistral Nebula consisting of an emission nebula of glowing gas (IC
2599) surrounding the small open cluster of stars (NGC 3324). Above the image
center is the larger star cluster NGC 3293, while to its right is the emission
nebula Loden 153.  The most famous occupant of the Carina Nebula, however, is
not shown. Off the image to the lower right is the bright, erratic, and doomed
star known as Eta Carinae -- a star once one of the brightest stars in the sky
and now predicted to explode in a supernova sometime in the next few mill

...

---
Title: The Great Globular Cluster in Hercules
Date: 2023-09-01
Explanation: In 1716, English astronomer Edmond Halley noted, "This is but a little Patch,
but it shows itself to the naked Eye, when the Sky is serene and the Moon
absent." Of course, M13 is now less modestly recognized as the Great Globular
Cluster in Hercules, one of the brightest globular star clusters in the northern
sky. Sharp telescopic views like this one reveal the spectacular cluster's
hundreds of thousands of stars. At a distance of 25,000 light-years, the cluster
stars crowd into a region 150 light-years in diameter. Approaching the cluster
core, upwards of 100 stars could be contained in a cube just 3 light-years on a
side. For comparison, the closest star to the Sun is over 4 light-years away.
The remarkable range of brightness recorded in this image follows stars into the
dense cluster core.


---
Title: A Desert Eclipse
Date: 2023-10-01
Explanation: A good place to see a ring-of-fire eclipse, it seemed, would be from a desert.
In a desert, there should be relatively few obscuring clouds and trees.
Therefore late December of 2019, a group of photographers traveled to the United
Arab Emirates and Rub al-Khali, the largest continuous sand desert in world, to
capture clear images of an unusual eclipse that would be passing over.  A ring-
of-fire eclipse is an annular eclipse that occurs when the Moon is far enough
away on its elliptical orbit around the Earth so that it appears too small,
angularly, to cover the entire Sun. At the maximum of an annular eclipse, the
edges of the Sun can be seen all around the edges of the Moon, so that the Moon
appears to be a dark spot that covers most -- but not all -- of the Sun. This
particular eclipse, they knew, would peak soon after sunrise.  After seeking out
such a dry and barren place, it turned out that some of the most interesting
ec

---
Title: Annular Solar Eclipse over Utah
Date: 2023-11-01
Explanation: Part of the Sun disappeared earlier this month, but few people were worried. The
missing part, which included the center from some locations, just went behind
the Moon in what is known as an annular solar eclipse.  Featured here is an
eclipse sequence taken as the Moon was overtaking the rising Sun in the sky. The
foreground hill is Factory Butte in Utah, USA. The rays flaring out from the Sun
are not real -- they result from camera aperture diffraction and are known as
sunstar. The Moon is real, but appears only in silhouette in this ring-of-fire
eclipse. As stunning as this eclipse sequence is, it was considered just
practice by the astrophotographer.  The reason? She hopes to use this experience
to better photograph the total solar eclipse that will occur over North America
on April 8, 2024.   Apply today (USA): Become a NASA Partner Eclipse Ambassador
Eclipse Album: Selected images sent in to APOD


---
Title: Milky Way Rising
Date: 2023-12-01
Explanation: The core of the Milky Way is rising beyond the Chilean mountain-top La Silla
Observatory in this deep night skyscape. Seen toward the constellation
Sagittarius, our home galaxy's center is flanked on the left, by the European
Southern Observatory's New Technology Telescope which pioneered the use of
active optics to accurately control the shape of large telescope mirrors. To the
right stands the ESO 3.6-meter Telescope, home of the exoplanet hunting HARPS
and NIRPS spectrographs. Between them, the galaxy's central bulge is filled with
obscuring clouds of interstellar dust, bright stars, clusters, and nebulae.
Prominent reddish hydrogen emission from the star-forming Lagoon Nebula, M8, is
near center. The Trifid Nebula, M20, combines blue light of a dusty reflection
nebula with reddish emission just left of the cosmic Lagoon. Both are popular
stops on telescopic tours of the galactic center. The composited image is a
stack of sep

---
